<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso3/ciclo1/2_semma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1li4ahmMhPo2cEUVqQKRDA9ahHp2py4Xb" width="100%">

# Sample, Explore, Modify, Model, Assess
---

En este notebook veremos una introducción práctica a la metodología estadística _Sample, Explore, Modify, Model, Assses_ (SEMMA), la cual se puede ver descrita en el siguiente diagrama:

<img src="https://drive.google.com/uc?export=view&id=1NB8u_CC-NmUyKBPiQ-9VFUi_8CdrJZ6q" width="80%">

Para la aplicación de machine learning usaremos dos librerías muy conocidas como `pandas`, para manipulación de datos, y `sklearn`, para modelamiento:

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from IPython.display import display

No obstante, usaremos una librería para visualización que es muy común en herramientas estadísticas (por ejemplo en el lenguaje de programación `R`):

In [ ]:
!pip install plotnine

## **1. Introducción a Plotnine**
---

`plotnine` es una librería que implementa [the layered grammar of graphics](https://byrneslab.net/classes/biol607/readings/wickham_layered-grammar.pdf). Se trata de una sintaxis general para definir gráficos estadísticos de una forma sencilla y personalizable.

<img src="https://drive.google.com/uc?export=view&id=1Fmyhpw_RojLlCnlfoUAozI-HZa-MBM5j" width="60%">

`plotnine` por detrás utiliza `matplotlib` para generar gráficas, de hecho podemos ver una comparativa con sus librerías hermanas:

- `matplotlib`: altamente personalizable pero requiere mucho código.
- `seaborn`: poco personalizable pero requiere muy poco código.
- `plotnine`: intermedio entre `seaborn` y `matplotlib`, es decir, no requiere tanto código y permite cierto grado de personalización.

_The layered grammar of graphics_ se conoce popularmente por el paquete `ggplot` de `R`, no obstante, con `plotnine` lo podemos usar desde _Python_. La creación de una gráfica con esta metodología consiste en dividir un gráfico estadístico en 7 capas:

<img src="https://drive.google.com/uc?export=view&id=1ZUblbDVFBTFQqvPm0-GXE8zD_njKtVAa" width="50%">

Estas capas nos permiten personalizar gráficos a distintos niveles:

- **Datos**: consiste en especificar la fuente de datos para los gráficos.
- **Aestética**: permite especificar qué elementos serán visibles en la gráfica, por ejemplo, qué va en el eje `x` y qué va en el eje `y`.
- **Geometrías**: permite especificar el tipo de gráfico que generaremos, por ejemplo, diagramas de barras, nubes de puntos, entre otros.
- **Facetas**: permite dividir el gráfico en distintos paneles (equivalente a los `subplots` de `matplotlib`).
- **Estadísticos**: permiten realizar operaciones estadísticas sobre los datos.
- **Coordenadas**: permite modificar los ejes coordenados de la visualización (coordenadas polares, escalas logarítmicas, entre otros).
- **Tema**: permite personalizar paletas de colores, fuentes del texto, entre otros.

Veamos un ejemplo sencillo de `plotnine` antes de continuar con la parte metodológica, primero importamos las funciones que necesitamos:

In [ ]:
from plotnine import ggplot, geom_point, aes, stat_smooth, facet_wrap
from plotnine.data import mtcars

Veamos el conjunto de datos de ejemplo que provee `plotnine`:

In [ ]:
display(mtcars)

[Motor Trend Car Road Tests](https://www.rdocumentation.org/packages/datasets/versions/3.6.2/topics/mtcars) (mtcars) es un conjunto de datos que contiene información del consumo de combustible en 10 aspectos relacionados al diseño y desempeño de 32 automóviles. Tenemos las siguientes variables:

- `mpg`: millas por galón.
- `cyl`: número de cilindros.
- `disp`: desplacamiento.
- `hp`: caballos de fuerza.
- `drat`: relación del eje trasero o diferencial.
- `wt`: peso.
- `qsec`: tiempo de cuarto de milla.
- `vs`: tipo de motor (0 = motor en v, 1 = motor recto).
- `am`: tipo de transmisión (0 = automático, 1 = manual).
- `gear`: número de engranajes delanteros.

Podemos generar una gráfica con `plotnine` para ver relaciones entre el peso y las millas por galón. Veamos cómo se genera:

In [ ]:
fig = (
        ggplot(mtcars, aes(x="wt", y="mpg", color="factor(gear)")) +
        geom_point() +
        stat_smooth(method="lm") +
        facet_wrap("~gear")
        )
display(fig)

En este caso:

- Definimos un objeto `ggplot` especificando los datos `mtcars` y la aestética `aes`.
- Con `"factor(gear)"` convertimos la columna `gear` a categorías.
- `geom_point` define una geometría de nube de puntos (`scatter`).
- `stat_smooth` define que se debe mostrar un modelo lineal `lm` sobre los datos.
- `facet_wrap` define que la gráfica se debe dividir en varios paneles de acuerdo a lo que contenga el campo `gear`.

Esta librería nos será de mucha ayuda para definir visualizaciones estadísticas en la metodología _SEMMA_, vamos a importar las funciones que necesitaremos de `plotnine`:

In [ ]:
from plotnine import (
        ggplot, aes, facet_wrap, geom_density,
        geom_tile, geom_boxplot, theme, element_text
        )

## **2. Contexto**
---

En este caso implementaremos una metodología para encontrar distintos perfiles de contaminación de productos alimenticios, para ello utilizaremos el conjunto de datos [food product emissions en Kaggle](https://www.kaggle.com/amandaroseknudsen/foodproductemissions).

<img src="https://drive.google.com/uc?export=view&id=1m3RV-gtqiaugZ5xvOBRwt9KmOcV7Brct" width="80%">

Para algunos de los productos alimenticios más comunes en todo el mundo, se ha encontrado que las emisiones de gases de efecto invernadero (GEI) producidos puede tener distintas causas que van desde la cadena de valor de los alimentos, cambio de uso de la tierra o LUC, hasta el comercio minorista (punto de compra/adquisición del usuario final).

Algo importante a tener en cuenta es que hay variaciones en las emisiones generadas de un mismo producto y distintos productos. Esto se debe fundamentalmente a factores como la región geográfica y el perfil ecológico del sistema de producción, el tamaño y el tipo de sistema de producción, entre otros. Este conjunto de datos utiliza la media global de GEI por tipo de alimento, esto con el fin de permitir un análisis más robusto.

Las emisiones de GEI se miden en kg de CO2 por kg de alimentos producidos.

> `kg CO2` = Kilogramo de dióxido de carbono.
    
Cargamos el conjunto de datos:

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/mindlab-unal/mlds6-datasets/main/u1/food_emissions.csv", index_col=0)
display(data.head())

El conjunto de datos tiene los siguientes campos:

- `product`: nombre del alimento.
- `land`: cambio de uso en la tierra.
- `feed`: emisión en alimentos (para animales).
- `farm`: emisión en granja.
- `processing`: emisión en procesamiento del producto.
- `transport`: emisión por transporte.
- `packaging`: emisión por empaquetado.
- `retail`: emisión por venta en tiendas minoristas.

## **3. Sample**
---

En la etapa de muestreo o _sample_ debemos seleccionar la muestra sobre la que realizaremos el estudio y determinar las variables de interés para el problema.

<img src="https://drive.google.com/uc?export=view&id=1ynp-FLIFxK7sJBArOxoPPChdig3vo5fb" width="80%">

En este caso, como es un conjunto de datos pequeño (43 registros) podemos usar la totalidad de las muestras, no obstante, vamos a diferenciar las variables numéricas de las categóricas:

In [ ]:
numeric_variables = [
        "land", "feed", "farm", "processing",
        "transport", "packaging", "retail"
        ]
category_variables = ["product"]

## **4. Explore**
---

La etapa de exploración consiste en encontrar descriptivos de las variables, determinar correlaciones y distribuciones.

<img src="https://drive.google.com/uc?export=view&id=1vRAmSKl03nSoCVOk5GLQoBdXX7G7c8Vz" width="80%">

Comenzamos generando descriptivos de las variables numéricas:

In [ ]:
display(data[numeric_variables].describe())

Con esto, nos hacemos a una idea de la escala de las variables. De aquí identificamos la necesidad de escalar los datos, debido a las diferencias de magnitudes entre distintas variables.

Ahora, vamos a visualizar la distribución de las variables. Para ello crearemos un gráfico de kernel de densidad con la librería `plotnine`.

Para generar la gráfica realizamos el siguiente proceso:

- Pivoteamos el dataframe con la función `melt` para que las variables de interés queden normalizadas (tabla vertical donde la selección de columnas se hace con un filtro y no por nombre de columna).
- Agregamos una aestética indicando que tendremos la magnitud de cada variable en el eje `x` y diferenciaremos entre campos por color.
- Agregamos un gráfico de kernel de densidad sin leyenda.
- Dividimos en distintos paneles de acuerdo a el tipo de variable.

Veamos el código:

In [ ]:
fig = (
        ggplot(
            data.pipe(
                pd.melt,
                id_vars = category_variables,
                value_vars = numeric_variables
                ),
            aes(x = "value", color = "variable")
            ) +
        geom_density(show_legend = False) +
        facet_wrap("~variable")
        )
display(fig)

También podemos visualizar la matriz de correlación entre las variables numéricas. Para generar la gráfica debemos realizar el siguiente proceso:

- Calculamos la matriz de correlación con el método `corr` y lo pivoteamos con la función `melt`.
- Agregamos una aestética indicando qué valores están en el eje `x` y cuáles en el eje `y`. El parámetro `fill` indica que vamos a colorear usando la magnitud de los datos.
- Con `geom_tile` mostramos puntos en el espacio en forma de rectángulos.
- Agregamos un tema especificando que el texto del eje `x` debe rotar 90 grados.

Veamos la gráfica:

In [ ]:
fig = (
        ggplot(
            (
                data
                .filter(numeric_variables)
                .corr()
                .reset_index()
                .pipe(pd.melt, id_vars = ["index"], value_vars = numeric_variables)
                ),
            aes(x = "index", y = "variable", fill = "value")
            ) +
        geom_tile() +
        theme(axis_text_x = element_text(rotation = 90))
        )
display(fig)

Podemos ver que la variable `farm` tiene una correlación alta (mayor a 0.5) con  `land` y `processing`, esto tiene sentido debido a que muchos de los productos de granja requieren cierto uso de tierra y tratamiento posterior.

## **5. Modify**
---

La etapa de modificación consiste en la limpieza y re-escalamiento de los datos.

<img src="https://drive.google.com/uc?export=view&id=1lfYVk_gnF5O88-O0R2sFycBqk4EMox4-" width="80%">

En este caso, realizaremos un _Z-scaling_ de las variables numéricas para que queden con media 0 y desviación estandar 1 por medio del `StandardScaler` de `sklearn`:

In [ ]:
transformer = StandardScaler()
features = transformer.fit_transform(data.filter(numeric_variables))
label = data.filter(category_variables)

Con esto, obtenemos una matriz numérica `features` con las siguientes dimensiones:

In [ ]:
display(features.shape)

También obtenemos el listado de productos sobre el que haremos un análisis posterior:

In [ ]:
display(label.head())

## **6. Model**
---

La etapa de modelamiento consiste en la definición y ajuste de un modelo.

<img src="https://drive.google.com/uc?export=view&id=1s69sNpUxMQ6CzR0r7_GJsFZBBuAObHWT" width="80%">

En este caso, para determinar perfiles de alimentos por emisiones usaremos el modelo `KMeans`. Adicional a esto, deseamos encontrar 8 perfiles de productos para su posterior análisis (en caso de no saber cuántos clusters o perfiles deseamos obtener, debemos hacer una exploración con métricas como el coeficiente de silueta o el índice de Davies-Bouldin).

In [ ]:
model = (
        KMeans(n_clusters = 8, random_state=42)
        .fit(features)
        )

## **7. Assess**
---

Por último, en la etapa de evaluación debemos analizar los resultados obtenidos y calcular indicadores clave de rendimiento (KPI) en caso de ser posible.

<img src="https://drive.google.com/uc?export=view&id=1iL-OtG_aOWxmBwOZ-t_96IZwmTrDGmiY" width="80%">

La evaluación suele ir acompañada con un diseño experimental para evaluar los modelos entrenados y determinar su aplicabilidad en el mundo real. En nuestro caso, haremos un análisis más descriptivo para interpretar los 8 perfiles de alimentos encontrados y sus distribuciones.

Para ello, vamos a crear un nuevo `DataFrame` con las características que obtuvimos y los clusters encontrados:

In [ ]:
new_data = (
        pd.DataFrame(
            data = features,
            columns = numeric_variables
            )
        .assign(
            clusters = model.predict(features),
            product = label
            )
        )
display(new_data)

Podemos generar una visualización de la distribución de las variables en cada uno de los perfiles encontrados, para ello realizamos el siguiente proceso:

- Normalizamos el conjunto de datos con la función `melt`.
- Agregamos una geometría de tipo diagrama de cajas y bigotes.
- Dividimos en distintos paneles por cluster o perfil.
- Agregamos un tema con el texto del eje `x` rotado.

Veamos la gráfica:

In [ ]:
fig = (
        ggplot(
            new_data.pipe(pd.melt, value_vars = numeric_variables, id_vars = "clusters"),
            aes(x = "variable", y = "value", color = "factor(clusters)")
            ) +
        geom_boxplot() +
        facet_wrap("~clusters") +
        theme(axis_text_x = element_text(rotation = 90))
        )
display(fig)

Con esto, podemos ver que hay perfiles como:

- Cluster 2 con niveles altos de alimentación `feed` y `retail`
- Cluster 5 con niveles altos de transporte.
- Cluster 7 con niveles bajos de tierra y empaquetado.

Podemos ver de una forma más clara los productos asociados a cada perfil por medio de una matriz de contingencia, para generarla debemos seguir el siguiente proceso:

- Generamos la matriz de contingencia entre los clusters encontrados y los nombres de los alimentos con la función `crosstab`.
- Normalizamos los datos con la función `melt`.
- Especificamos el eje `x` para los perfiles y el eje `y` para el tipo de alimento con una aestética.
- Agregamos un gráfico por rectángulos con `geom_tile`.
- Agregamos un tema con el texto del eje `x` rotado.

Veamos la gráfica:

In [ ]:
cross = pd.crosstab(new_data["clusters"], new_data["product"])
categories = cross.columns
fig = (
        ggplot(
            (
                cross
                .reset_index()
                .pipe(pd.melt, id_vars = ["clusters"], value_vars = categories)
                ),
            aes(x = "clusters", y = "product", fill = "value")
            ) +
        geom_tile() +
        theme(axis_text_x = element_text(rotation = 90))
        )
display(fig)

De esta gráfica podemos ver patrones en los perfiles:

- Cluster 5 contiene productos derivados del azúcar como `Beet Sugar` o `Cane Sugar`.
- Cluster 3 contiene productos similares en producción al aceite como `Sunflower Oil`, `Soybean Oil` o `Rapeseed Oil`.
- Cluster 0 contiene productos que necesitan tierra para producirse como frutas, verduras o legumbres.

Estos perfiles pueden ser de gran utilidad para interpretación de un conjunto de datos y para estudios un poco más estadísticos. No obstante, a pesar de que muchas etapas de la metodología _SEMMA_ tienen su equivalente con otras metodologías de ciencia de datos, su enfoque es más estadístico.

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [Sklearn](https://scikit-learn.org/)
- [Plotnine](https://plotnine.readthedocs.io/en/stable/)

## Créditos
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Asistente docente**:

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Diseño de imágenes:**

- [Brian Chaparro Cetina](mailto:bchaparro@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*